In [ ]:
import pandas as pd
import numpy as np
from scipy import stats
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from tqdm.notebook import tqdm
import re
pd.set_option('max_columns', None)
pd.options.display.max_colwidth = 100

In [ ]:
df_groupby_user = pd.read_csv('data_groupby_user.csv', index_col=0)
df_groupby_user.shape

In [ ]:
df = pd.read_csv('data.csv', index_col=0)
df.shape

In this notebook we analyze agent churn from different perspectives.

# Calculate user activity

In [ ]:
# keep only interested columns
df2 = df[['agent','bytes_returned']].copy()
# convert to datetime
df2['datetime'] = pd.to_datetime(df['timestamp'])
df2 = df2.set_index('datetime')
df2.head()

In [ ]:
# groupby agent and downsample into 1 hour bins
df2 = df2.groupby('agent').resample('60T').count()
df2 = df2.drop(columns=['agent'])
df2 = df2.reset_index()
df2 = df2.rename(columns={"datetime": "timestamp","bytes_returned": "request_count"})
# remove data from 01-03
df2 = df2[df2['timestamp'].dt.day==2]
# get hour from datetime
df2['hour'] = df2['timestamp'].dt.hour
df2 = df2.drop(['timestamp'], axis=1)
df2.shape

In [ ]:
df2.head()

In [ ]:
# pivot dataframe
df3 = df2.pivot(index='agent', columns='hour', values='request_count')
# fill NaN and change type to int
df3 = df3.fillna(0)
df3 = df3.astype(int)
df3.shape

In [ ]:
df3.head()

In [ ]:
df3.to_csv('data_user_activity.csv') # 4.5MB

# Request heatmap

We draw a heatmap of the number of requests made by each agent in each hour. The x axis represents the agents, sorted in the order that they first appeared in the dataset. The y axis represents the hours of the day. The color of each cell in the heatmap indicates the number (in natural logarithmic scale) of requests made by a specific agent during a specific hour.

In [ ]:
df4 = pd.read_csv('data_user_activity.csv', index_col=0)
df4.shape

In [ ]:
# sort by the order users first appear in the data
df_agent = pd.DataFrame(df['agent'].unique())
df_agent.columns = ['agent']
df4 = df4.reindex(index=df_agent['agent'])
df4 = df4.reset_index()
# drop users with no request in the day
df4 = df4[~df4.isna().any(axis=1)]
# convert to int
df4 = df4.set_index('agent')
df4 = df4.astype(int)
df4 = df4.reset_index()
df4.shape

In [ ]:
df4.set_index('agent',inplace=True)
# df4[df4 > 1] = 1
df4 = np.log(df4)
df4 = df4.replace(float('-inf'), 0)
df4 = df4.reset_index()

df4 = df4.reset_index()
df4['index'] = df4['index']/df4.shape[0]
df4 = df4.set_index('index')

df4.head()

In [ ]:
data = df4.drop(['agent'],axis=1)
data = data.T

fig = px.imshow(data, color_continuous_scale='turbo', origin='lower',
               labels=dict(color="number of request<br>an agent made per hour<br>(log scale)"),)

fig.update_xaxes(side='bottom')
fig.update_xaxes(title="agent (sorted by first occurrence)", tickformat = ',.0%')
fig.update_yaxes(title="hour")

fig.show()

# Churn rate

Next, we look into the percentage of users who stay in the network after a specific time period.

In [ ]:
# keep only interested columns
df3 = df[['agent']].copy()
# convert to datetime
df3['datetime'] = pd.to_datetime(df['timestamp'])
df3.head()

In [ ]:
df4 = df3.groupby('agent').agg(['min', 'max', 'count'])
df4.columns = df4.columns.get_level_values(1)
df4[df4['count']==1].sum()

In [ ]:
df4['last'] = df4['max'] - df4['min']
df4['minute'] = df4['last'].dt.seconds/60
df4['minute'] = df4['minute'].astype(int)
df4 = df4.drop(['min','max','last','count'],axis=1)
df4.reset_index()
df4.head()

In [ ]:
df5 = pd.DataFrame(df4.value_counts(ascending=False))
df5 = df5.reset_index()
df5.columns = ['minute','count']
df5.head()

In [ ]:
df4.shape

In [ ]:
df6 = pd.DataFrame(columns = ['stay_after_minute', 'count', 'name', 'hour'])

def addRow(df6, m, name):
    c = df5[df5['minute'] >= m].sum()['count']
    df6 = df6.append({'stay_after_minute' : m, 'count' : c, 'name':name, 'hour':m/60},
        ignore_index = True)
    return df6

In [ ]:
df6 = addRow(df6, 1, "1m")
df6 = addRow(df6, 2, "2m")
df6 = addRow(df6, 4, "4m")
df6 = addRow(df6, 8, "8m")
df6 = addRow(df6, 16, "16m")
df6 = addRow(df6, 60, "1h")
df6 = addRow(df6, 120, "2h")
df6 = addRow(df6, 240, "4h")
df6 = addRow(df6, 480, "8h")
df6 = addRow(df6, 960, "16h")
df6 = addRow(df6, 1439, "24h")

In [ ]:
df6['percentage'] = df6['count']/21985
# df6['stay_after_minute'] = df6['stay_after_minute'].astype(str)

In [ ]:
df6.head()

In [ ]:
fig = px.line(df6, x='hour', y='percentage', 
             text = [str(x[0])+' {0:1.2f}%'.format(x[1]*100) for x in zip(df6['name'],df6['percentage'])])

fig.update_traces(textposition="middle right")

fig.update_xaxes(title='duration since agent made first request (hour)')
fig.update_yaxes(title='percentage of agent stays online', tickformat=',.0%')

fig.show()

# Users leave in 1 min

Next, we only keep the users who disconnect from the network within a minute and count the number of requests they made.

In [ ]:
df4.head()

In [ ]:
df9 = df4[df4['minute']<1]
df9.shape

In [ ]:
df9 = df9.reset_index()
agent_set = df9['agent'].unique()

In [ ]:
df10 = df[df['agent'].isin(agent_set)]
df10.shape

In [ ]:
df.shape

In [ ]:
df10.head()

In [ ]:
df11_1 = df10[['agent','cid']].groupby('agent').agg(['count',pd.Series.nunique])
df11_1.columns = df11_1.columns.get_level_values(1)
df11_2 = df10[['agent','bytes_returned']].groupby('agent').agg('mean')
df11 = df11_1.join(df11_2, lsuffix='agent', rsuffix='agent')
df11['bytes_returned'] = df11['bytes_returned']/1024
df11 = df11.rename(columns={"count": "cid_count", "nunique": "cid_unique", "bytes_returned": "KB_returned_mean"})
df11.head()

In [ ]:
df11.describe()

In [ ]:
df11 = df11.sort_values(by=['cid_count','cid_unique'], ascending=False)
df11 = df11.reset_index()
df11 = df11.reset_index()
df11.head()

In [ ]:
df12 = pd.DataFrame(columns = ['request', 'count'])

def addRow(df12, l, r):
    df_temp = df11[(df11['cid_count'] >= l) & (df11['cid_count'] < r)]
    c = df_temp.count()[0]
    df12 = df12.append({'request':'['+str(l)+','+str(r)+')', 'count':c}, ignore_index = True)
    return df12

df12 = addRow(df12, 1, 2)
df12 = addRow(df12, 2, 10)
df12 = addRow(df12, 10, 100)
df12 = addRow(df12, 100, 1000)
df12 = addRow(df12, 1000, 10000)

df12 = df12.replace('[1,2)', '1')
df12 = df12.replace('[1000,10000)', '[1000,+∞)')

total = df12['count'].sum()
df12['percentage'] = df12['count']/total
df12

In [ ]:
fig = px.bar(df12, x='request', y='count', text=[str(x[0])+'\n{0:1.2f}%'.format(x[1]*100) for x in zip(df12['count'],df12['percentage'])])

fig.update_xaxes(title='Range of request count per agent (left within a minute)')
fig.update_yaxes(title='agent count')
                                                                 
fig.show()